* https://github.com/h2oai/h2ogpt

Modified: 2 Oct 2024, Jon Chun

# Document Question-Answer [h2oGPT](https://github.com/h2oai/h2ogpt)

In this notebook, we demonstrate how one can use h2oGPT with a large language model.

To begin, please get free ngrok account to get auth token (e.g.) using your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup .  You will be asked for this token below in an input box.

In [1]:
%%time

# NOTE: 21s

!git clone https://github.com/h2oai/h2ogpt.git
!cd h2ogpt && git checkout 2668694581347b0d1afe76760213db46f7214126 -q
!cp -ar h2ogpt/. ./
!rm -r h2ogpt

Cloning into 'h2ogpt'...
remote: Enumerating objects: 40043, done.
remote: Counting objects: 100% (1512/1512), done.
remote: Compressing objects: 100% (552/552), done.
remote: Total 40043 (delta 1000), reused 1295 (delta 960), pack-reused 38531 (from 1)
Receiving objects: 100% (40043/40043), 53.13 MiB | 4.58 MiB/s, done.
Resolving deltas: 100% (28402/28402), done.


In [2]:
%%time

# NOTE: 32s

# Install pyhon 3.10 that will be used within pipenv
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [3]:
%%time

# NOTE:

# Install dependencies
!for fil in requirements.txt reqs_optional/requirements_optional_langchain.txt reqs_optional/requirements_optional_gpt4all.txt reqs_optional/requirements_optional_langchain.gpllike.txt reqs_optional/requirements_optional_langchain.urls.txt ; do pip install -r $fil ; done


  Cloning https://github.com/huggingface/peft.git (to revision 0b62b4378b4ce9367932c73540349da9a41bdea8) to /tmp/pip-req-build-m0y_k1gi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-m0y_k1gi
  Running command git rev-parse -q --verify 'sha^0b62b4378b4ce9367932c73540349da9a41bdea8'
  Running command git fetch -q https://github.com/huggingface/peft.git 0b62b4378b4ce9367932c73540349da9a41bdea8
  Running command git checkout -q 0b62b4378b4ce9367932c73540349da9a41bdea8
  Resolved https://github.com/huggingface/peft.git to commit 0b62b4378b4ce9367932c73540349da9a41bdea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Ignoring pypandoc: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%%time

# NOTE: ~8m

# Sign-up for free ngrok account using (e.g.) your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup

!pip install pyngrok
import getpass
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open an http ngrok tunnel
connection_string = ngrok.connect(7860, "http").public_url
print("Once server is up and says Running on local URL:  http://0.0.0.0:7860, click on this link, then click on Visit Site: %s" % connection_string)

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
Once server is up and says Running on local URL:  http://0.0.0.0:7860, click on this link, then click on Visit Site: https://f621-104-199-189-124.ngrok-free.app


In [ ]:
%%time

# NOTE:

# To create a public link, set `share=True` in `launch()`

# !GRADIO_SERVER_PORT=7860 python generate.py --base_model=togethercomputer/RedPajama-INCITE-Chat-3B-v1 --prompt_type=human_bot --score_model=None --langchain_mode=ChatLLM --visible_langchain_modes="['ChatLLM', 'UserData', 'MyData']" --user_path=user_path --share=False --hf_embedding_model=sentence-transformers/all-MiniLM-L6-v2
!GRADIO_SERVER_PORT=7860 python generate.py --base_model=togethercomputer/RedPajama-INCITE-Chat-3B-v1 --prompt_type=human_bot --score_model=None --langchain_mode=ChatLLM --visible_langchain_modes="['ChatLLM', 'UserData', 'MyData']" --user_path=user_path --share=True --hf_embedding_model=sentence-transformers/all-MiniLM-L6-v2

Using Model togethercomputer/redpajama-incite-chat-3b-v1
Prep: persist_directory=db_dir_ChatLLM does not exist, regenerating
Prep: persist_directory=db_dir_UserData does not exist, regenerating
0it [00:00, ?it/s]
0it [00:00, ?it/s]
Loaded 0 sources for potentially adding to UserData
2024-10-02 13:25:56.095552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 13:25:56.123618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 13:25:56.132540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 13:25:57.872231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:3

In [8]:
# can kill old ngrok + generate and try again
do_kill = False
if do_kill:
  !pkill -f generate --signal 9
  !pkill -f frpc_linux_amd --signal 9
  !pkill -f ngrok --signal 9